# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV 

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
733,HOUSEHOLD_1_173_CA_1_validation,HOUSEHOLD_1_173,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,5,12,10,4,2,1,5,2,1
739,HOUSEHOLD_1_179_CA_1_validation,HOUSEHOLD_1_179,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,2,1,0,...,0,2,1,2,3,2,1,1,0,4
764,HOUSEHOLD_1_204_CA_1_validation,HOUSEHOLD_1_204,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,4,3,7,5,1,0,9,8,3,9
813,HOUSEHOLD_1_254_CA_1_validation,HOUSEHOLD_1_254,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,4,8,6,0,7,3,5,3,8,7
835,HOUSEHOLD_1_277_CA_1_validation,HOUSEHOLD_1_277,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,1,0,2,0,0,1,0,1,2,0
1003,HOUSEHOLD_1_448_CA_1_validation,HOUSEHOLD_1_448,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,16,14,11,3,4,1,0,7,6,5
1014,HOUSEHOLD_1_459_CA_1_validation,HOUSEHOLD_1_459,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,3,1,0,2,2,5,2,2,5,3
1020,HOUSEHOLD_1_465_CA_1_validation,HOUSEHOLD_1_465,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,24,21,15,10,9,10,13,16,37,24
3716,HOUSEHOLD_1_106_CA_2_validation,HOUSEHOLD_1_106,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA,0,0,0,0,...,0,6,7,3,2,2,1,3,4,2
3727,HOUSEHOLD_1_118_CA_2_validation,HOUSEHOLD_1_118,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA,8,4,4,3,...,4,7,5,5,2,6,4,2,6,6


# User-defined functions to calculate Metrics

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [ ]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
data_pattern = data_cleaning(sales_erratic)

In [ ]:
data_pattern

,Date,HOUSEHOLD_1_024_CA_3,HOUSEHOLD_1_106_CA_2,HOUSEHOLD_1_118_CA_2,HOUSEHOLD_1_147_CA_2,HOUSEHOLD_1_151_CA_2,HOUSEHOLD_1_151_CA_4,HOUSEHOLD_1_173_CA_1,HOUSEHOLD_1_173_CA_2,HOUSEHOLD_1_179_CA_1,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,2.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,0.0,0.0,0.0,4.0,3.0,3.0,3.0,6.0,3.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,0.0,0.0,0.0,2.0,1.0,3.0,1.0,4.0,3.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,0.0,0.0,0.0,2.0,3.0,1.0,6.0,0.0,4.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,0.0,0.0,0.0,5.0,11.0,10.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,0.0,0.0,0.0,3.0,1.0,3.0,1.0,0.0,4.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,0.0,0.0,0.0,0.0,4.0,2.0,6.0,9.0,2.0,...,0,0,0,1,0,0,0,1,0,0


# Random Forest Hyperparameters to find optimum parameters

In [ ]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [ ]:
end_train_date = '2016-04-10'

In [ ]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [ ]:
correlation_threshold = 0.8

In [ ]:
n_estimators = [30, 50, 100, 150]
max_depth = list(range(4,9)) + [None]
min_samples_leaf = range(4,11)
max_features = range(20,30)
bootstrap = [True] #We force bootstrap
max_samples = [0.6, 0.7, 0.8, 0.9]

In [ ]:
# User-defined funtion to find optimum parameters on training data
def Random_Forest_Hyperparameters_Tuning(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators, max_depth, min_samples_leaf,
                          max_features, bootstrap, max_samples):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_hyper_params = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        param_dist = {'n_estimators': n_estimators,
                      'max_depth': max_depth,
                      'min_samples_leaf': min_samples_leaf,
                      'max_features': max_features,
                      'bootstrap': bootstrap,
                      'max_samples': max_samples}

        forest = RandomForestRegressor(n_jobs=1)

        forest_cv = RandomizedSearchCV(forest, param_dist, cv=5, n_jobs=-1, verbose=2, n_iter=300, scoring='neg_mean_absolute_error')
        forest_cv.fit(X_train,y_train)
            
        best_params_temp = forest_cv.best_params_
        
        df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
        df_output_temp['Product'] = product
        
        df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)
        
    return df_output_hyper_params

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
list_pattern_sample = sample(list_erratic, 30)

In [ ]:
list_pattern_sample

['HOUSEHOLD_1_179_CA_1_validation',
 'HOUSEHOLD_1_204_CA_3_validation',
 'HOUSEHOLD_1_318_CA_2_validation',
 'HOUSEHOLD_2_342_CA_3_validation',
 'HOUSEHOLD_1_118_CA_2_validation',
 'HOUSEHOLD_1_389_CA_3_validation',
 'HOUSEHOLD_1_251_CA_3_validation',
 'HOUSEHOLD_1_418_CA_2_validation',
 'HOUSEHOLD_2_389_CA_2_validation',
 'HOUSEHOLD_1_204_CA_1_validation',
 'HOUSEHOLD_1_506_CA_2_validation',
 'HOUSEHOLD_1_151_CA_2_validation',
 'HOUSEHOLD_1_254_CA_1_validation',
 'HOUSEHOLD_1_235_CA_3_validation',
 'HOUSEHOLD_1_334_CA_2_validation',
 'HOUSEHOLD_1_465_CA_2_validation',
 'HOUSEHOLD_1_277_CA_2_validation',
 'HOUSEHOLD_1_173_CA_2_validation',
 'HOUSEHOLD_1_459_CA_2_validation',
 'HOUSEHOLD_1_430_CA_3_validation',
 'HOUSEHOLD_2_499_CA_2_validation',
 'HOUSEHOLD_1_173_CA_1_validation',
 'HOUSEHOLD_1_521_CA_2_validation',
 'HOUSEHOLD_1_465_CA_1_validation',
 'HOUSEHOLD_1_459_CA_1_validation',
 'HOUSEHOLD_1_024_CA_3_validation',
 'HOUSEHOLD_1_277_CA_1_validation',
 'HOUSEHOLD_1_303_CA_2_valid

In [ ]:
start = timer()

df_hyper_params = Random_Forest_Hyperparameters_Tuning(list_pattern_sample, not_id_cols, lags, moving_averages, stds,
                                                       maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                                       end_train_date, n_estimators, max_depth, min_samples_leaf,
                                                       max_features, bootstrap, max_samples)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOUSEHOLD_1_179_CA_1
Progressing: 0.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_204_CA_3
Progressing: 3.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_318_CA_2
Progressing: 6.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_2_342_CA_3
Progressing: 10.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_118_CA_2
Progressing: 13.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_389_CA_3
Progressing: 16.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_251_CA_3
Progressing: 20.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_418_CA_2
Progressing: 23.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Curren

In [20]:
df_hyper_params

,n_estimators,min_samples_leaf,max_samples,max_features,max_depth,bootstrap,Product
0,100,7,0.7,21,7,True,HOUSEHOLD_1_179_CA_1
1,30,10,0.9,25,4,True,HOUSEHOLD_1_204_CA_3
2,30,7,0.7,22,6,True,HOUSEHOLD_1_318_CA_2
3,30,10,0.8,22,5,True,HOUSEHOLD_2_342_CA_3
4,100,8,0.6,26,7,True,HOUSEHOLD_1_118_CA_2
5,50,4,0.8,25,7,True,HOUSEHOLD_1_389_CA_3
6,30,6,0.8,29,4,True,HOUSEHOLD_1_251_CA_3
7,150,9,0.6,26,6,True,HOUSEHOLD_1_418_CA_2
8,30,7,0.7,21,None,True,HOUSEHOLD_2_389_CA_2
9,30,9,0.6,21,4,True,HOUSEHOLD_1_204_CA_1


In [21]:
df_hyper_params.to_csv('Random_Forest_Hyperparams_Erratic.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [22]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_hyper_params.groupby(['n_estimators', 'min_samples_leaf', 'max_samples', 
                         'max_features', 'max_depth', 'bootstrap'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators min_samples_leaf max_samples max_features max_depth bootstrap         
30           4                0.6         28           5         True             1
             10               0.8         22           5         True             1
150          7                0.7         24           8         True             1
100          10               0.8         20           8         True             1
             8                0.6         26           7         True             1
             7                0.7         21           7         True             1
50           10               0.9         21           4         True             1
                              0.8         29           6         True             1
             8                0.8         29           8         True             1
             7                0.8         29           6         True             1
             6                0.6         23           7         True             1
             4                0.8         25           7         True             1
                              0.6         27           7         True             1
30           10               0.9         25           4         True             1
                              0.7         22           8         True             1
             6                0.8         21           5         True             1
             9                0.8         21           8         True             1
                              0.7         29           6         True             1
                              0.6         28           6         True             1
                                          24           8         True             1
                                          21           4         True             1
                                          20           5         True             1
             8                0.9         28           8         True             1
                              0.7         22           6         True             1
                              0.6         22           7         True             1
             7                0.9         26           6         True             1
                              0.7         22           6         True             1
             6                0.8         29           4         True             1
150          9                0.6         26           6         True             1

In [23]:
# See what value of parameter "n_estimators" occurs the most frequently
df_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,18
50,7
100,3
150,2


In [24]:
# See what value of parameter "min_samples_leaf" occurs the most frequently
df_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,3
6,3
7,6
8,5
9,7
10,6


In [25]:
# See what value of parameter "max_samples" occurs the most frequently
df_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,10
0.7,7
0.8,9
0.9,4


In [26]:
# See what value of parameter "max_features" occurs the most frequently
df_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
20,2
21,6
22,5
23,1
24,2
25,2
26,3
27,1
28,3


In [27]:
# See what value of parameter "max_depth" occurs the most frequently
df_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
4,4
5,4
6,8
7,6
8,7


# Implement Random Forest on Test Set with optimum set of parameters

In [28]:
n_estimators_chosen = 30
min_samples_leaf_chosen = 6
max_samples_choose = 0.6
max_features_choose = 23
max_depth_choose = 8
bootstrap = [True]

In [29]:
# User-defined function to run Random Forest on Test Data

def Random_Forest_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                          max_samples_choose, max_features_choose, max_depth_choose, bootstrap):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_rf_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        forest = RandomForestRegressor(n_jobs=1,
                                      n_estimators = n_estimators_chosen,
                                      min_samples_leaf = min_samples_leaf_chosen,
                                      max_samples = max_samples_choose,
                                      max_features = max_features_choose,
                                      max_depth = max_depth_choose,
                                      bootstrap = bootstrap,
                                      random_state = 0)

        forest.fit(X_train,y_train)
        y_pred = forest.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_rf_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [30]:
# Start running Random Forest on Test Set

start = timer()

df_forecast_values = Random_Forest_Test_Data(list_erratic, not_id_cols, lags, moving_averages, stds,
                                             maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                             end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                                             max_samples_choose, max_features_choose, max_depth_choose, bootstrap)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOUSEHOLD_1_024_CA_3
Progressing: 0.0 %
Currently Running Product: HOUSEHOLD_1_106_CA_2
Progressing: 2.5 %
Currently Running Product: HOUSEHOLD_1_118_CA_2
Progressing: 5.0 %
Currently Running Product: HOUSEHOLD_1_147_CA_2
Progressing: 7.5 %
Currently Running Product: HOUSEHOLD_1_151_CA_2
Progressing: 10.0 %
Currently Running Product: HOUSEHOLD_1_151_CA_4
Progressing: 12.5 %
Currently Running Product: HOUSEHOLD_1_173_CA_1
Progressing: 15.0 %
Currently Running Product: HOUSEHOLD_1_173_CA_2
Progressing: 17.5 %
Currently Running Product: HOUSEHOLD_1_179_CA_1
Progressing: 20.0 %
Currently Running Product: HOUSEHOLD_1_204_CA_1
Progressing: 22.5 %
Currently Running Product: HOUSEHOLD_1_204_CA_3
Progressing: 25.0 %
Currently Running Product: HOUSEHOLD_1_235_CA_3
Progressing: 27.5 %
Currently Running Product: HOUSEHOLD_1_251_CA_3
Progressing: 30.0 %
Currently Running Product: HOUSEHOLD_1_254_CA_1
Progressing: 32.5 %
Currently Running Product: HOUSEHOLD_1_254_CA_2
Prog

In [31]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,0.0,0.000573,HOUSEHOLD_1_024_CA_3
2016-04-12,0.0,0.000573,HOUSEHOLD_1_024_CA_3
2016-04-13,0.0,0.000573,HOUSEHOLD_1_024_CA_3
2016-04-14,0.0,0.003123,HOUSEHOLD_1_024_CA_3
2016-04-15,0.0,0.002667,HOUSEHOLD_1_024_CA_3
...,...,...,...
2016-04-20,0.0,0.000513,HOUSEHOLD_2_499_CA_2
2016-04-21,0.0,0.020513,HOUSEHOLD_2_499_CA_2
2016-04-22,0.0,0.000513,HOUSEHOLD_2_499_CA_2


In [32]:
df_forecast_values.to_csv('Random Forest_Erratic_Forecast_Values.csv')

In [33]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [34]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0027091410333317375 minutes


In [35]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3,inf,inf,200.0000,NaN,0.001236,0.001626
1,HOUSEHOLD_1_106_CA_2,inf,inf,200.0000,NaN,0.008355,0.008626
2,HOUSEHOLD_1_118_CA_2,1.117109,93.3394,126.0341,104.6715,3.866916,4.740711
3,HOUSEHOLD_1_147_CA_2,0.648026,38.7708,46.9241,41.7025,1.246203,1.504977
4,HOUSEHOLD_1_151_CA_2,0.791584,66.5996,67.5573,96.3081,3.044555,3.866122
5,HOUSEHOLD_1_151_CA_4,0.673426,65.0846,64.8527,74.5274,1.813070,2.553299
6,HOUSEHOLD_1_173_CA_1,0.753545,57.1063,67.9537,78.3172,3.304005,5.214066
7,HOUSEHOLD_1_173_CA_2,0.903882,74.2867,102.4057,39.5569,2.016353,2.725731
8,HOUSEHOLD_1_179_CA_1,0.675606,59.1155,68.6635,37.0495,1.351211,1.706048
9,HOUSEHOLD_1_204_CA_1,0.823706,81.4654,83.7498,113.8916,2.851290,4.769366


In [36]:
df_forecast_values.to_csv('Random Forest_Erratic_Forecast_Metrics.csv')